# 🍏 A*pple Quest  
**Artificial Intelligence Course Project**  
**Authors:** Alessandro Querci, Andrea Lepori  
**Academic Year:** 2024/2025

# Introduction 
This notebook constitutes the report for the final project of the AIF course, project's name is "A*pple Quest" and our group is composed by Alessandro Querci and Andrea Lepori. The goal of this project is to explore classical Artificial Intelligence **search and planning algorithms**, among the ones seen during the course, within the MiniHack environment, a reinforcement learning platform built on top of NetHack. Our work started by trying to identify a task suitable to apply the methodologies and algorithms seen in class (GOFAI, no learning). In particular, we decided to focus on the use of state space search and planning algorithms. The high-level idea is to design some custom rooms/environments containing apples (with associated reward) and a custom reward manager. The reward manager assigns a positive reward for each collected apple and a (typically larger) reward when the agent reaches the downstairs by successfully completing the task. Additionally, to incentivize the agent to be as efficient as possible in terms of the number of steps, a (small) constant penalty is assigned for each step the agent moves through. Then, to evaluate the different search algorithms, they are used to generate a path to complete the task (reaching the downstairs) while maximizing the reward. We decided to test the algotithms both in an *offline setting* with *full observability* and in an **online setting** with **partial observability**. The environment is determinstic, discrete, static, episodic and single agent. As mentioned above, if we are in the offline setting the observability is full, in the online setting the observability is partial. To test our algorithms we defined different rooms/environments/mazes with different complexity. We started with a simple rectangular room, named "simple room", with some random apples and no obstacles, to start testing and debugging the algorithms, then we add obstacles like lava in the so-called "lava room" that allow more difficult tasks while preserving the full observability. Then we introduce walls that reduce the visibility of the agent as you can see in the "simple_maze.des" and in the more difficult "complex_maze.des". To compare and evaluate each algorithm the more natural metric is the **total reward** collected by the agent accomplishing the task and computing automatically by the Reward Manager, however we decide to evaluate the algorithms across other metrics: **success rate** (task-completion/reaching the downstairs), **planning time** (time to elaborate a path/plan), **path length** (number of steps in the path), **apples collected** (numbers of collected apples during the path). Once the algorithms were written and the environments defined, a model selection was carried out using grid search to optimize some of the hyperparameters. For all algorithms, given the same environment, several runs/simulations were executed with randomized apple positions, starting points, and downstairs, and the average of the metrics of interest was calculated in order to compare the algorithms across various dimensions and evaluate the trade-offs in using one over another.

# Related works
To realize our project, we relied heavily on MiniHack. 
> "MiniHack is a sandbox framework for easily designing rich and diverse environments for Reinforcement Learning (RL). Based on the game of NetHack, MiniHack uses the NetHack Learning Environment (NLE) to communicate with the game and to provide a convenient interface for customly created RL training and test environments of varying complexity." 

> — https://github.com/samvelyan/minihack

In particular, we studied and heavily used the APIs provided by Minihack to customize our environment, customize the reward manager, manage the simulation, etc...

We took also ispiration, as a starting point, from the hands-on sessions of the course and from some projects of previous students like "Get The Gold!" by Federico Cioni, Claudia Gentili and Giovanni Guerrazzi (https://github.com/fefex302/AIF_minihack_project/) that have applied search algorithms but to a different task and to a different environment. In particular they defined a customized Minihack environment, the gold room, essentially a square or rectangular room filled with some pieces of gold and some leprechauns. The aim of their agent is to reach the exit stairs, maximizing the gain with the leprechauns that act as opponents because their main goal is to take all the gold they manage. In particular they studied scenarios with full observability and non deterministic. In our work, apart from focusing on a different task and environment, we extend the application of the search algorithms to environment with partial observability but we restrict to completely deterministic environment. 

# Methodologies 
To implement our project we use Python as programming language, in particular you can find all the algorithms implementation in files .py, as well as the implementation of the various simulators. To test our implementations, compare algorithms we use Jupyter notebooks. In the rest of this section, I will present briefly the two distinct approaches we follow, as well as the algorithms, in the **offline** setting and in the **online setting**. 
First, a shared operation for the two settings is the **env generation**. Precisely, we leverage level generator package in MiniHack to customize the environment, first by generating the .des file of a map with a function create_map, specifiying with a specific string the map layout  and possible presence of lava or walls. Then we add the starting position where the agent is spawned, the downstairs, and the apple positions, in random positions with the possibility to specificy a seed to ensure repitibility. Then with a function create_env we effectively create the env with the specified map and reward settings. The user can specify the reward to assign to apple collection as well as the penalty time for each step taken. In this function we define the set of actions that the agent is allowed to perform (to move and to pickup and eat the apples), we define the reward manager to register the specified rewards and penalty. Then using gym.make we effectively create the env.     

## Offline

### Simulator offline
The simulator for the offline setting is implemented by the function **simulate_offline_planning**, that takes in input the environment to simulate and the pathfinding function to use for simulation (to compute the path). In practice, the simulator retrieves from the game map the position of the player, the target, and the apples in the environment, and makes a call to the pathfinding algorithm that returns the path found. The path, expressed as a sequence of coordinates, is translated into a sequence of actions that the agent must perform. Subsequently, the path is executed with closed eyes in perfect offline mode. For each action in the list of actions, the function env.step is called. If you land on an apple, a series of actions is performed in order to eat it, and then you continue on your path. During the sequence of actions, the overall reward is accumulated by collecting the various instantaneous rewards. The task is successfully completed if the downstairs are reached. 

### Algorithms offline 
At the core of the offline setting task there are the pathfinding algorithms that are called to determine the path the agent has to execute. The implementations of all the used algorithms can be found in algorithms.py regarding search algorithms in state spaces, while the implementation of Monte Carlo Tree Search can be found in the MCTS.py file. Now a brief description of the implemented algorithms, for more details please refer to the code: 

**a_star_apple**: This function implements a modified A* pathfind algorithm to find an optimal path from a start position to a target, in addition to the standard A* components (g-scores for actual cost, heuristic estimates, and a priority queue), the algorithm introduces a reward system for paths that pass near or through apples. Apples directly reduce the movement cost (g-score), with a stronger bonus for stepping onto one and a smaller bonus for being adjacent. This encourages the pathfinder to favor apple-rich routes without compromising its goal-directed behavior. A weight parameter allows tuning the influence of the heuristic, effectively enabling weighted A* behavior.

**a_star_collect_apples**: This function implements an A* pathfinding variant that aims to collect all apples on the grid before reaching the target. It uses a Minimum Spanning Tree (MST)-based heuristic to estimate the minimal cost required to visit all remaining apples and the target, combining this with the actual path cost (g) for A*'s total cost function (f = g + h). It prioritizes paths that visit uncollected apples efficiently, balancing exploration and optimization using a tunable weight parameter on the heuristic (weighted A*). 

**potential_field_path**: This simple algorithm, born in the field of robotics, uses a potential field approach to guide pathfinding toward a target while encouraging apple collection along the way. It assigns each grid position a scalar "potential" based on its attractive force toward the target and any remaining apples, using either the sum or maximum of these forces (modality_potential). Random noise is added to break ties and avoid local minima, and repeated visits to the same cell are penalized (repulsive attraction) to discourage loops. The agent greedily chooses the next move that maximizes total potential.

**mcts** : This algorithm applies Monte Carlo Tree Search (MCTS) to navigate the state space. Each node in the search tree represents a game state defined by the agent’s position and the set of collected apples.
At each iteration, the algorithm follows the standard four MCTS phases:
- __Selection__: Nodes are traversed using the UCB1 formula to balance exploration and exploitation.
- __Expansion__: New child nodes are added by simulating valid moves from the current state.
- __Simulation__ (Rollout): A random policy simulates play from the new state to a terminal condition or step limit, assigning a reward based on apples collected and whether the target is reached.
- __Backpropagation__: The simulation result is propagated up the tree to update visit counts and cumulative rewards.
The best path is then extracted by following the most visited child nodes from the root.

**greedy_best_first_search**: This algorithm uses Greedy Best-First Search to find a path from a start point to a target while collecting all apples on a grid. It prioritizes nodes based solely on a heuristic estimate of the cost to reach the goal, ignoring the actual path cost. The heuristic evaluates the shortest possible path through all remaining apples and then to the target, using either Manhattan or cached BFS distances.

**beam_search_apple**: This algorithm applies Beam Search to find a path from a start point to a target while collecting apples for additional reward. At each step, it keeps only the top k candidate paths (defined by the beam width) that maximize a net score: total apple reward minus path cost. All pairwise paths between the start, apples, and target are precomputed using A* for efficiency. The search explores new candidates by extending paths toward unvisited apples or the target, using the precomputed paths, avoiding cycles.

## Online 

### Simulator Online 
The online simulation, in a partially observable environment, is implemented by the function simulate_online. At the heart of the simulator's logic is a while loop that, until the episode is finished, takes the current position of the agent in the game and calculates a path through an online pathfinding algorithm. The identified path is converted into a series of actions that the agent begins to execute. Being in a partially observable environment, the agent has full visibility only within a limited radius around itself. If during the execution of the current plan the agent discovers the presence of a new apple, it stops and recalculates a new plan, performing a new iteration of the loop. Likewise, if the agent exhausts the actions of the current plan, a new iteration of the loop leads to the computation and  identification of a new path. If during execution the agent is on a cell with an apple, it performs all the necessary actions to collect and eat it.

### Algorithms Online 
It is clear that a fundamental part of the online simulation algorithm is the online pathfinding algorithm. In file algorithms_online.py it is possible to find the implementation of path planning algorithms online. 

__a_star_online__: This algorithm implements an online version of the A* search strategy tailored for interactive environments where goals may change. It dynamically plans a path from the current position to the nearest uncollected apple. If no apples remain, it instead plans a path to the exit stairs. With the repeated call to this function the agent re-evaluates and replans at each step using the A* algorithm with the Manhattan distance as the heuristic.

__planner_online__: A more general online planner for navigating a partially known map, that takes any pathfinding function (planner_func) as input (any of those implemented for the offline setting A*. MCTS, Greedy search,...). It first identifies the current goals (apples and stairs) by calling find_target, then plans a path accordingly. __find_target__ determines the current navigation target given the game map and player position: it locates all apples and attempts to find the stairs. If stairs are unavailable, searches for frontier tiles, known walkable tiles adjacent to unexplored (unknown) areas, to enable map exploration and sets as a target the frontier tile that maximize a frontier score function. If no frontier or stairs are found but apples exist, defaults to the first apple as target.
Returns both the set of apple positions and the chosen target. __frontier_search__ function scans the map to identify frontier tiles (tiles that are currently known and accessible but border unknown (unexplored) spaces). This helps the agent prioritize exploring new areas. __score_frontier__: Assigns a score to each frontier tile based on both the length of the path from the current position (shorter is better).The amount of adjacent unknown tiles (higher is better, representing information gain). This scoring guides the agent to explore frontiers that maximize new information per movement cost.

# Assessment 
Two distinct benchmarks were performed, one for the offline setting and another for the online setting, respectively in the notebooks Benchmark_Offline.ipynb and BenchMark_Online.ipynb. The metrics used for the benchmarking are: 
- __total reward__: at the end the better agent is the one which maximizes the custom reward. 
- __success rate__: the agent is able to accomplish the task (i.e reaching downstairs)?
- __apples collected__: how many apples the agent eats? How biased is the algorithm toward collecting apples?
- __path lenght__: How long is the path in term of number of steps. 
- __planning time__: How long does the pathfinding algorithm take to calculate the path? The less, the better.


## Results 
### Offline benchmark on "lava env", best averaged result in terms of Reward for each algorithm
| Algorithm   | Avg Reward | Avg Path Length | Avg Apples | Avg Success Rate | Avg Planning Time |
|-------------|------------|-----------------|------------|------------------|-------------------|
|A* Star      |  1.065	   |      34.7       |    5.0     |        1.0       |      0.004        |
|MCTS         |  0.715     |      27.4       |    3.1     |        1.0       |      2.33         |
|Greedy BFS   |  0.85      |      35.0       |    5.0     |        1.0       |      0.05         |
|Pot. Fields  |  0.745     |      34.1       |    4.7     |        1.0       |      0.04         |
|Beam Search  |  0.055	   |      29.3       |    2.9     |        1.0	     |      0.09         |

### Online benchmark on "simple maze", best averaged result in terms of Reward for each algorithmn 
| Algorithm | Avg Reward | Avg Path Length | Avg Apples | Avg Success Rate | Avg Planning Time |
|-----------|------------|-----------------|------------|------------------|-------------------|
|A* Star    | 4.822      | 84.8            | 4.4        | 1.0              | 0.08              |
|MCTS       | 4.954      | 77.1            | 4.5        | 1.0              | 0.87              |
|Greedy BFS | 5.351      | 81.9            | 4.9        | 1.0              | 0.07              |
|Pot. Fields| 4.776      | 131.1           | 4.8        | 1.0              | 0.082             |
|Beam Search| 4.390      | 73.8            | 3.9        | 1.0              | 0.18              |

### Online benchmark on "complex maze", best averaged result in terms of Reward for each algorithmn 
| Algorithm   | Avg Reward | Avg Path Length | Avg Apples | Avg Success Rate | Avg Planning Time |
|-------------|------------|-----------------|------------|------------------|-------------------|
| A* Star     | 3.684      | 139.2           | 3.8        | 1.0              | 0.23              |
| MCTS        | 2.729      | 219.1           | 3.7        | 1.0              | 5.18              |
| Greedy BFS  | 3.911      | 139.7           | 4.0        | 1.0              | 0.35              |
| Pot. Fields | 3.345      | 147.5           | 3.6        | 1.0              | 0.16              |
| Beam Search | 2.636      | 155.5           | 2.9        | 1.0              | 1.269             |



# Conclusion 

# Appendix

## Cached BFS heuristic 
The cached_bfs heuristic enhances pathfinding in grid environments with obstacles by computing the shortest path length between two points using a breadth-first search (BFS) that respects walls and barriers. To improve efficiency, it caches previously computed distances to avoid redundant searches. This heuristic provides an accurate distance estimate that accounts for impassable terrain, making it well-suited for environments where direct straight-line metrics (like Manhattan distance) are insufficient due to obstacles.

## Project Structure

- `Benchmark_Offline.ipynb` — Notebook for offline benchmark experiments
- `Benchmark_Online.ipynb` — Notebook for online benchmark experiments  
- `MCTS.py` — Implementation of Monte Carlo Tree Search   
- `algorithms.py` — Search & Planning algorithms
- `algorithms_online.py` — Online search algorithms 
- `complex_maze.des` — Complex maze environment description file  
- `simple_maze.des` — Simple maze environment 
- `simulator.py` — Implementation of the various simulators
- `utils.py` — Utility functions 
- `Report.ipynb` — Project Report (only Text/Markdown)

## Team contributions
The project work was carried out by the two members of the group, in particular:
- **Alessandro Querci**: implemented the logic of the simulators, a_star_apple, beam search, online algorithms, frontier search and benchmarking 
- **Andrea Lepori**: implemented a_star_collect_apples, MCTS, potential fields, Greedy Best First, wrote the report.

The work on the project involved some meetings and pair programming sessions as well as individual work using github as a collaboration platform.

## Relationship with the course
The work of this project has strong relations with the topics of the AIF course, in particular with the section on state space search algorithms. Particular attention was paid to stick to the typology of classical AI algorithms studied in the course (no machine learning). However, we tried to experiment with algorithms not seen during the course such as Potential fields or lightly touched upon such as A* online or with heuristics specifically designed for our task such as bfs_cache. Furthermore, our work has a strong relationship with the practical lessons held during the hands-on sessions.
